# FE MINI PROEJCT: DS CHAMPS: WEB SCRAPING

Team :Anusha Nagineni, Dharmendra Dash, Adiseshulu Pujari

###### Project Description:
We are web scraping Daily Essential products from 3 e-commerce websites (DMart, Grofers and BigBasket) and provide best recommendation for the products.  

###### Below 4 tasks will be performed in this project.
 
    A. Data Acquisition  
    B. Data Cleaning 
    C. Data Integration 
    D. Exploratory Data Analysis and Recommendation

A. Data Acquisition : We fetch product data from e-commerce websites using Web Scraping. 

Steps followed to Web Scrape:
 1.  Identify products that we want to scrap from our chosen 3 online e-commerce platforms.
 2.  Search the products on the browser using API URLs to fetch product info.
 3.  Understand elements in the response from the webiste API call to get product info.
 4.  Program to fetch product details like name, quantity, price, discountprice if any and save the data to csv files.

     

In [15]:
#Necessary Library Imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import requests 


###### Products and Categories to be scrapped

In [7]:
df_products = pd.read_csv("products.csv" )
print("Total categories to be scrapped ", len(df_products["Product"].unique()))  
print("Total products to be scrapped ", df_products.shape[0])  

Total categories to be scrapped  30
Total products to be scrapped  142


##### Scrapping DMart

 - We need to pass the page number to the the URL for DMart.
 - We are limiting scrapting to 2 pages
 - DMart requires a Store Id be sent in the header to be sent. Otherwise, it will return wrong results

In [24]:
#Dmart Scrapping
def findProducts_dmart(search_brand, search_product):
    #Max pages to scrap
    max_pages = 2
    #Product full name
    searchStr = search_brand + " " + search_product
    #URL for page 
    url_page  = "https://digital.dmart.in/api/v1/search/{0}?page={1}"
    #Necessary request headers
    headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}
    
    #DMart requires store Id 
    headers['storeId']='10657'
    #Replace spaces in the product name with +
    search_str_encoded = searchStr.replace(" ","+")
    
    #Extract product data into a dictionary object
    product_data=[]
    for page in range(1,max_pages):
        url = url_page.format(search_str_encoded, page)
        #print(url)
        productsInfo = requests.get(url, headers=headers).json()
        #If the scrapping did not yield any products, just continue with next product
        products=productsInfo.get( 'suggestionView')
        if(not products):
            continue
     
        for product in  products:
            skus = product['skus']
            for sku in skus:  
                if(sku['defining']):
                    name =   sku.get( 'name',  product['name']).strip()
                    product_data.append({"search_brand": search_brand, "search_product": search_product, "shop": "DMart",   "product_name": name,"weight": sku['defining'][0]['volume'], "mrp": sku['price_MRP'], "special_price": sku['price_SALE'] })
    #Return the scrapped data           
    return product_data

##### Scrapping Grofers
 - We need to pass the page number to the the URL for Grofers.
 - Grofers requires a special header to be sent
 - We are limiting scrapting to 2 pages


In [25]:
#Grofers Scrapping
def findProducts_grofers(search_brand, search_product):
    #Max pages to scrap
    max_pages = 2
    #Product full name
    searchStr = search_brand + " " + search_product
    #URL for page. This should include Longitude and Latitude as well. 
    url_page  = "https://grofers.com/v5/search/merchants/26659/products/?lat=17.4196281427546&lon=78.3790778036223&q={0}&suggestion_type=0&t=1&start={1}&size=48"
    #Necessary request headers
    headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}
    #Grofers requires this header 
    headers['app_client'] = 'consumer_web' 
    #Replace spaces in the product name with +
    search_str_encoded = searchStr.replace(" ","+")
    #Extract product data into a dictionary object
    product_data=[]
    for page in range(1,max_pages):
        start_pos = (page-1)*48
        if(start_pos==0):
            start_pos = 1
        #Create page specific URL
        url = url_page.format(search_str_encoded, start_pos)
        productsInfo = requests.get(url, headers=headers).json()
        #If the scrapping did not yield any products, just continue with next product
        products= productsInfo.get('products')
        if(not products):
            break
        for product in  products:
            prod_variants = product['variant_info']
            for var in prod_variants:
                product_data.append({"search_brand": search_brand, "search_product": search_product,   "shop": "Grofers",  "product_name": var["line_1"],"weight": var["unit"], "mrp": var["mrp"], "special_price": var["price"]})
    #Return the scrapped data    
    return product_data

##### Scrapping Bigbasket

- We need to maintain two separate URLs in the code as Big basket has different URLS for first page and other pages.
- We are limiting scrapting to 2 pages


In [26]:
#Big Basket Scrapping
def findProducts_bigbasket(search_brand, search_product):
    
    #Max pages to scrap
    max_pages = 2
    
    #Product full name
    searchStr = search_brand + " " + search_product
    
    #URL for page 1
    url_page_1 = "https://www.bigbasket.com/custompage/getsearchdata/?type=deck&slug={0}"
    
    #URL for subsequent pages
    url_for_page_n = "https://www.bigbasket.com/product/get-products/?slug={0}&page={1}&tab_type=[%22all%22]&sorted_on=relevance&listtype=ps"
    
    #Replace spaces in the product name with +
    search_str_encoded = searchStr.replace(" ","+")
    
    #Form actual URL for page 1
    url = url_page_1.format(search_str_encoded)
 
    #Necessary request headers
    headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}
    
    #Get the response for page 1
    productsInfo = requests.get(url, headers=headers).json()
    
    #Extract product data into a dictionary object
    products =  productsInfo['json_data']['tab_info'][0]['product_info']['products']
    product_data=[]
    for product in  products:
        options = product['all_prods']
        if(not options): 
            product_data.append({"search_brand": search_brand, "search_product": search_product, "shop": "Big Basket", "product_name": str(product['p_brand']).strip() + " " + str(product['p_desc']).strip(), "weight":product['w'], "mrp": product['mrp'], "special_price": product['sp'] })
            continue
        for option  in product['all_prods']:
            product_data.append({"search_brand": search_brand, "search_product": search_product, "shop": "Big Basket",  "product_name": str(product['p_brand']).strip() + " " + str(product['p_desc']).strip(), "weight":option['w'],
                                 "mrp": option['mrp'], "special_price": option['sp'] }) 
 
    
    #Do it for subsequent pages     
    for page in range(2,max_pages):
        newurl = url_for_page_n.format(search_str_encoded, page) 
        productsInfo = requests.get(newurl, headers=headers).json()
        products =  productsInfo['tab_info']['product_map']['all']['prods']

        for product in  products:
            options = product['all_prods']
            if(not options): 
                product_data.append({"search_brand": search_brand, "search_product": search_product, "shop": "Big Basket",    "product_name": str(product['p_brand']).strip() + " " + str(product['p_desc']).strip(), "weight":product['w'], "mrp": product['mrp'], "special_price": product['sp'] })
                continue
            for option  in product['all_prods']:
                product_data.append({"search_brand": search_brand, "search_product": search_product,  "shop": "Big Basket",  "product_name": str(product['p_brand']).strip() + " " + str(product['p_desc']).strip(), "weight":option['w'], "mrp": option['mrp'], "special_price": option['sp'] })

    #Return the scrapped data           
    return product_data
 

###### Do Web Scrapping for all 3 websites

In [34]:
# Scrapping for all sites
def scrapeAll():
    #Read all items for which scrapping is needed
    df_input   = pd.read_csv("products.csv" )
    #Define an empty dataframe to store results for each of the shops
    column_names =   ["search_brand","search_product", "shop",   "product_name", "weight", "mrp", "special_price"]
    
    df_bigbasket = pd.DataFrame(columns = column_names)
    df_dmart = pd.DataFrame(columns = column_names)
    df_grofers = pd.DataFrame(columns = column_names)
    
    #For each of the products, peform scrapping
    for ( idx , search_brand, search_product) in df_input.itertuples():
        print('Scrapping DMart for ', idx, search_brand, search_product)
        product_data = findProducts_dmart(search_brand, search_product)
        df_dmart = df_dmart.append(product_data[:3])
        
        print('Scrapping Grofers for ', idx, search_brand, search_product)
        product_data = findProducts_grofers(search_brand, search_product)
        df_grofers = df_grofers.append(product_data[:3])   
        
        print('Scrapping BigBasket for ', idx, search_brand, search_product)
        product_data = findProducts_bigbasket(search_brand, search_product)
        df_bigbasket = df_bigbasket.append(product_data[:3])

       
    #Store final results for each of the stores in separate csv file
    df_dmart.to_csv('dmartDailyEssentails.csv', index=False, encoding='utf-8')
    df_grofers.to_csv('grofersDailyEssentails.csv', index=False, encoding='utf-8')
    df_bigbasket.to_csv('big_basketDailyEssentails.csv', index=False,encoding='utf-8')

###### Scrapping Results

In [33]:
#Call scrapping routine
scrapeAll()
#Load scrapped data into data frames and look at the metrics
df_dmart  = pd.read_csv("dmartDailyEssentails.csv")
df_grofers  = pd.read_csv("grofersDailyEssentails.csv")
df_bb  = pd.read_csv("big_basketDailyEssentails.csv")

print("DMart returned products count:" , df_dmart.shape[0] )
print("Grofers returned products count:" , df_grofers.shape[0] ) 
print("Bigbasket returned products count:" , df_bb.shape[0] )


Scrapping BigBasket for  0 3 Roses Tea
Scrapping Grofers for  0 3 Roses Tea
Scrapping BigBasket for  0 3 Roses Tea
Scrapping BigBasket for  1 7 Up Soft Drink
Scrapping Grofers for  1 7 Up Soft Drink
Scrapping BigBasket for  1 7 Up Soft Drink
Scrapping BigBasket for  2 ACT II Popcorn
Scrapping Grofers for  2 ACT II Popcorn
Scrapping BigBasket for  2 ACT II Popcorn
Scrapping BigBasket for  3 Adidas Deodrant
Scrapping Grofers for  3 Adidas Deodrant
Scrapping BigBasket for  3 Adidas Deodrant
Scrapping BigBasket for  4 Anurag Cooking Oil
Scrapping Grofers for  4 Anurag Cooking Oil
Scrapping BigBasket for  4 Anurag Cooking Oil
Scrapping BigBasket for  5 Appy Fizz Juice
Scrapping Grofers for  5 Appy Fizz Juice
Scrapping BigBasket for  5 Appy Fizz Juice
Scrapping BigBasket for  6 Aswini Hair Oil
Scrapping Grofers for  6 Aswini Hair Oil
Scrapping BigBasket for  6 Aswini Hair Oil
Scrapping BigBasket for  7 Bisleri Mineral Water
Scrapping Grofers for  7 Bisleri Mineral Water
Scrapping BigBasket f

IndexError: list index out of range

# B. Data Cleaning

The files generated by WebScraping has raw data. 
This data has to be cleaned by identiying below issues and fixing them and saving to new csv files.
    1. Removing Noise or special characters
    2. Missing Values Imputation
    3. Deriving Quantity from product descriptions and discount from MRP and SP.
    4. Removing Duplicate records.
    5. Arrange all columns in same order.



###### Cleaning up for DMart

In [40]:
df  = pd.read_csv("dmartDailyEssentails.csv",  engine='python')
print(df.product_name.unique())
print(df.weight.unique())

FileNotFoundError: [Errno 2] No such file or directory: 'dmartDailyEssentails.csv'

Below issues will be fixed on Dmart data.
 - There are products with combo packs. So, remove them as identifying individual product becomes tough during recommendation
 - Remove products with weights that has invalid characters other than numbers
 - Drop duplicate products that may have arrived due to wrong search results given by the site
 - Product name contains weight. Remove it. 

###### Cleaning up Big Basket

In [ ]:
df  = pd.read_csv("big_basketDailyEssentials.csv",  engine='python')
print(df.product_name.unique())
print(df.weight.unique())

Below issues will be fixed on BigBasket data.

 - There are products with combo packs. So, remove them as identifying individual product becomes tough during recommendation
 - Remove products with weights that has invalid characters other than numbers
 - Drop duplicate products that may have arrived due to wrong search results given by the site

In [ ]:
#Cleanup routine for bigbasket
def clean_bigbasket_data():
    #Read the dataset
    df  = pd.read_csv("big_basket.csv",  engine='python')
    # Pattern to identify combo packs
    searchfor = ['x', 'X','\+','Comb',  'each', '\(']
    #Drop the rows with above patterns
    df.drop( df[ df["weight"].str.contains('|'.join(searchfor)) ].index,inplace=True  )
    #Separate weight in to weight, measure
    df[['weight','measure']] = df.weight.str.split(expand=True) 
    #Remove rows that have invalid weight entries
    df.drop( df[~df["weight"].str.replace('.','',1).str.isnumeric() ].index,inplace=True  )
    #Check if product name has the keyword we searched for
    df['good_product_ind'] = [x[1] in x[0] for x in zip(df['product_name'], df['search_brand'])]
    #Drop the products that don;t meet the above criteria
    df.drop( df[~df["good_product_ind"] ].index,inplace=True  )   
    #Creae weight_measure column by combining weight and measure
    df[ 'weight_measure' ] = df.weight.str.strip() + df.measure.str.strip()
    #Re-order columns
    df = df[['search_product','search_brand', 'shop' ,'product_name','weight', 'measure', 'weight_measure', 'mrp','special_price']]
    #Calculate discount
    df['discount'] = (df.mrp - df.special_price) *100 / df.mrp 
    #Remove duplicate entries for given product and weight
    df.drop_duplicates(subset=['product_name','weight'], keep='last', inplace=True)
    #Save the results to big_basket_cleaned
    df.to_csv('big_basket_cleaned.csv', index=False)

###### Cleaning up for Grofers

In [ ]:
df  = pd.read_csv("grofers.csv",  engine='python')
print(df.product_name.unique())
print(df.weight.unique())

Below issues will be fixed for Grofers data

 - There are products with combo packs. So, remove them as identifying individual product becomes tough during recommendation
 - Remove products with weights that has invalid characters other than numbers
 - Drop duplicate products that may have arrived due to wrong search results given by the site

In [ ]:
#Cleanup routine for grofers
def clean_grofers_data():
    #Read the dataset
    df  = pd.read_csv("grofers.csv")
    # Pattern to identify combo packs
    searchfor = ['x', 'X', ',', 'Refills', 'Bags','Sachets','Drops','Wipes','U','Wipe','Pellets','Cubes','unit','units','tablets',"\+",'XL','L','M','S','Set','Pink']
    #Drop the rows with above patterns
    df.drop( df[ df["weight"].str.contains('|'.join(searchfor)) ].index,inplace=True  )
    #Separate weight in to weight, measure
    df[['weight','measure']] = df.weight.str.split(expand=True) 
    #Remove rows that have invalid weight entries
    df.drop( df[~df["weight"].str.replace('.','',1).str.isnumeric() ].index,inplace=True  )
    #Creae weight_measure column by combining weight and measure
    df[ 'weight_measure' ] = df.weight.str.strip() + df.measure.str.strip()
    #Check if product name has the keyword we searched for
    df['good_product_ind'] = [x[1] in x[0] for x in zip(df['product_name'], df['search_brand'])]
    #Drop the products that don;t meet the above criteria
    df.drop( df[~df["good_product_ind"] ].index,inplace=True  )    
    #Re-order columns
    df = df[['search_product','search_brand', 'shop' ,'product_name','weight', 'measure', 'weight_measure', 'mrp','special_price']]
    #Calculate discount
    df['discount'] = (df.mrp - df.special_price) *100 / df.mrp 
    #Remove duplicate entries for given product and weight
    df.drop_duplicates(subset=[  'product_name','weight'],keep='last', inplace=True)
    #Save the results to grofers_cleaned
    df.to_csv('grofers_cleaned.csv', index=False)


###### Cleaning up all datasets at once

In [42]:
#Run cleanup 
clean_dmart_data()
clean_grofers_data()
clean_bigbasket_data()

df_dmart  = pd.read_csv("dmart_cleaned.csv")
df_grofers  = pd.read_csv("grofers_cleaned.csv")
df_bb  = pd.read_csv("big_basket_cleaned.csv")

print("DMart products count after cleanup:" , df_dmart.shape[0] )
print("Grofers products count after cleanup:" , df_grofers.shape[0] ) 
print("Bigbasket products count after cleanup:" , df_bb.shape[0] )

NameError: name 'clean_dmart_data' is not defined

# C. Data Integration

Steps followed:
1. All the cleaned files from 3 websited will be loaded into 3 dataframes and will be concatenated into a single dataframe.
2. All products will be sorted on the product type and name.
3. The final dataframe is converted into csv for reference.

In [ ]:
#Combine cleaned files into combined.csv file
def integrate_products():
    #Load cleaned files into separate dataframes 
    df_dmart  = pd.read_csv("dmart_cleaned.csv")
    df_grofers  = pd.read_csv("grofers_cleaned.csv")
    df_bb  = pd.read_csv("big_basket_cleaned.csv")
   
    #Combine these files into a single data frame
    df_combined = pd.concat([df_bb, df_dmart, df_grofers])
    
    #Sort them based on searched product and resultant product name
    df_combined.sort_values(by=[ 'search_product' , 'product_name'], inplace=True)
    #Reset the index
    df_combined = df_combined.reset_index(drop=True)
    #Store the results in final file combined.csv
    df_combined.to_csv('combined.csv', index=False)
     
    return df_combined

In [ ]:
#Combine Products 
integrate_products()
#Read combined products into data frame
df_combined  = pd.read_csv("combined.csv")
print("Final Scrapped and Cleaned Product Count:" , df_combined.shape[0] )

# D. EXPLORATORY DATA ANALYSIS & RECOMMENDATION

Steps followed:
1. Final dataframe of combined csv and various graphs are plotted for vizualization and inferences are made.
2. Best Recommendation provided for a chosen product on the basis of least price by checking discount.

## EDA  


In [ ]:
#Read the file into a dataframe
items = pd.read_csv('combined.csv')

In [ ]:
#print shape of the combined dataset
print("The total count of products available across shops ",  items.shape[0])

In [ ]:
#print the first few rows
items.head()

In [ ]:
#Check if there are any null values in the final dataset
items.isnull().sum()

In [ ]:
#Which shops are part of the dataset
items.shop.unique()

In [ ]:
#Unique product categories
items.search_product.unique()

In [ ]:
#Scatter plot for all attributes
sns.pairplot(items)
plt.show()

In [ ]:
#Heatmap for all attributes
plt.figure(figsize = (10,8))
corr = items.corr()
sns.heatmap(corr, annot=True)
plt.show()



###### Inference: The above heat map shows there is a tight correlation between mrp and special_price.

In [ ]:
## How many products does each shop have to offer?
prods_shops_df = items[['shop','search_product']]
prod_shops_counts =prods_shops_df.groupby(['shop'],as_index =False).count()
prod_shops_counts.plot.bar(x="shop", y="search_product", rot=70, title="Products accross Vendors");
 

###### Inference: Big Basket has highest number of products available while Grofers has the least. Customers are most likely to find a product in Big Basket.

In [ ]:
#How many products avaialble for each product category overall
products_count =prods_shops_df.groupby(['search_product'],as_index =False).count()
dataFrame = pd.DataFrame(data=products_count);
y_pos = np.arange(len(dataFrame))
# Draw a vertical bar chart
ax = dataFrame.plot.bar(x="search_product", y="shop", rot=80, title="Products and their Availability");
ax.set_xlabel("Product")
ax.set_ylabel("Count")


In [ ]:
###  Number of products sold per product category by each shop 
pd.crosstab(items.shop, items.search_product, margins=True, margins_name="Total",rownames=['shop']).T

In [ ]:
#  Which Individual Products  have Discount
discounts_df = items[ (items.discount > 0.0)]
  
discounts_shop_products = discounts_df[['search_product','shop']]
discounts_per_shop =discounts_shop_products.groupby(['shop'],as_index =False).count()
ax = discounts_per_shop.plot.bar(x="shop", y="search_product", rot=70, title="Products accross Vendors with discount");
ax.set_xlabel("Shop")
ax.set_ylabel("Product count with discount") 
 

###### Inference: DMart offers the most discounts. So customers looking for discounts should visit DMart.

## Recommendation  


Ensure combined.csv file is present in current path before executing following recommendation code.

In [43]:
#Import necessary libraries for user interface
from ipywidgets import interact, Dropdown, HTML, Layout, Box, Label
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

#Read data into dataframe
df_combined = pd.read_csv("combined.csv")

#Unique product types that the user can buy
products_list = df_combined["search_product"].unique()     

#Create dropdown widgets for the user to interact
product_type_dropdown = Dropdown(description="Product Type:", options = products_list)
product_names_dropdown = Dropdown(description="Product:")
product_sizes_dropdown = Dropdown(description="Size:")
recommendation_html = HTML(
    value=" ",
    placeholder='',
    description='',  layout={'width': 'max-content'}
)

box = Box(
    [
        Label(value='Recommendation:'),
        recommendation_html
    ]
) 
#Event handler when product category changes
def update_product_name_options(change):  
    product_type = product_type_dropdown.value         
    df_product = df_combined[  df_combined.search_product.str.lower().str.contains(product_type.lower())    ]
    products = df_product["product_name"].unique()
    product_names_dropdown.options = products
product_type_dropdown.observe(update_product_name_options )
 
#Event handler when product name changes
def update_product_size_options(change): 
    if change['name'] == 'value' and (change['new'] != change['old']):
        product_type = product_type_dropdown.value
        product_name = product_names_dropdown.value
        df_product = df_combined[  df_combined.search_product.str.lower().str.contains(product_type.lower())    ]
        product_name = product_name.replace("+", "\+")
        df_product = df_product[  df_product.product_name.str.lower().str.contains(product_name.lower())   ] 
        product_sizes = df_product["weight_measure"].unique()
        product_sizes_dropdown.options = product_sizes
product_names_dropdown.observe(update_product_size_options )

#Event handler when product size changes
def recommend_product(change):
    if change['name'] == 'value' and (change['new'] != change['old']):
        product_type = product_type_dropdown.value
        product_name = product_names_dropdown.value
        product_size = product_sizes_dropdown.value
        recommendation_html.value=""
        product_name = product_name.replace("+", "\+")
        if(product_size):
            df_final_list = df_combined[ df_combined.search_product.str.lower().str.contains(product_type.lower()) & 
                                         df_combined.product_name.str.lower().str.contains(product_name.lower()) & 
                                         df_combined.weight_measure.str.lower().str.contains(product_size.lower()) ] 
            df_final_list.sort_values(by='special_price', inplace=True)
            discount = round(  df_final_list[ "discount"].iloc[0] , 2)
            recommendation_html.value= "Buy it from <b>" + df_final_list.shop.iloc[0] + "</b> for Rs."  + str(df_final_list[ "special_price"].iloc[0])  + " discount of " + str(discount) + "%"
product_sizes_dropdown.observe(recommend_product )

#Display all user interface elements 
display(product_type_dropdown)
display(product_names_dropdown)
display(product_sizes_dropdown)
display(box)
update_product_name_options(None )
    

FileNotFoundError: [Errno 2] File b'combined.csv' does not exist: b'combined.csv'